In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
import pickle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM, Flatten
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load Dataset

In [ ]:
names = ['Name',
         'Maths_1_10', 'Literature_1_10', 'Physics_1_10', 'Chemistry_1_10', 'Biology_1_10', 'History_1_10', 'Geography_1_10', 'English_1_10', 'Civic Education_1_10',
         'Maths_2_10', 'Literature_2_10', 'Physics_2_10', 'Chemistry_2_10', 'Biology_2_10', 'History_2_10', 'Geography_2_10', 'English_2_10', 'Civic Education_2_10',
         'Maths_1_11', 'Literature_1_11', 'Physics_1_11', 'Chemistry_1_11', 'Biology_1_11', 'History_1_11', 'Geography_1_11', 'English_1_11', 'Civic Education_1_11',
         'Maths_2_11', 'Literature_2_11', 'Physics_2_11', 'Chemistry_2_11', 'Biology_2_11', 'History_2_11', 'Geography_2_11', 'English_2_11', 'Civic Education_2_11',
         'Maths_1_12', 'Literature_1_12', 'Physics_1_12', 'Chemistry_1_12', 'Biology_1_12', 'History_1_12', 'Geography_1_12', 'English_1_12', 'Civic Education_1_12',
         'Maths_2_12', 'Literature_2_12', 'Physics_2_12', 'Chemistry_2_12', 'Biology_2_12', 'History_2_12', 'Geography_2_12', 'English_2_12', 'Civic Education_2_12']

In [ ]:
file = "/content/drive/MyDrive/NCKH_Review of Student's Performance/Data_Phat/10_11_12_header_removed.xlsx"
df = pd.DataFrame(pd.read_excel(file, header=None, sheet_name='10_11_12', names=names)).drop(columns = 'Name', axis=1)

In [ ]:
df.head()

,Maths_1_10,Literature_1_10,Physics_1_10,Chemistry_1_10,Biology_1_10,History_1_10,Geography_1_10,English_1_10,Civic Education_1_10,Maths_2_10,...,Civic Education_1_12,Maths_2_12,Literature_2_12,Physics_2_12,Chemistry_2_12,Biology_2_12,History_2_12,Geography_2_12,English_2_12,Civic Education_2_12
0,7.6,6.7,6.9,7.0,6.1,7.1,6.3,6.8,8.4,7.0,...,8.3,8.2,7.3,7.6,8.0,7.2,6.9,8.8,6.9,8.2
1,5.0,6.1,7.2,6.7,7.5,6.4,7.8,7.0,8.9,5.3,...,8.2,7.8,8.8,7.3,8.4,7.4,8.9,8.1,7.5,8.6
2,4.2,5.8,5.2,6.2,5.6,7.1,6.9,5.7,8.4,5.5,...,6.4,7.8,6.8,6.2,7.9,8.6,7.7,6.8,6.8,8.1
3,4.2,5.6,6.2,5.9,6.6,7.3,6.9,6.1,8.3,6.7,...,7.2,7.6,7.4,6.8,8.4,7.1,7.6,7.5,6.5,7.4
4,5.5,6.6,6.3,5.4,7.0,8.8,7.8,6.2,9.1,4.9,...,8.1,7.4,8.5,8.4,8.0,7.7,8.3,8.0,7.3,9.4


In [ ]:
df.shape #kích thước data: 1257 hang - 56 cot

(1257, 54)

In [ ]:
from keras.callbacks import History
history = History()


# Function

In [ ]:
def err(subject, actual, pred): #mae,mape,mse,rmse là các hàm mất mát (loss function), ở đây có tác dụng như công thức để tính toán độ chính xác cho hàm Accuracy bên dưới
#với subject là tên môn, actual là giá trị điểm thực sự (ytest), pred là giá trị điểm dự đoán(kq trả về sau khi model dự đoán - ypred),
#so sánh độ chênh lệnh của điểm dự đoán so với điểm thực sự sẽ tìm được độ chính xác
  mae = mean_absolute_error(actual, pred)
  mape = mean_absolute_percentage_error(actual, pred)
  mse = mean_squared_error(actual, pred)
  rmse = np.sqrt(mse)
  res = {'Subject': subject,'MAE': mae, 'MAPE (%)': mape*100, 'MSE': mse, 'RMSE': rmse, 'Accuracy (%)': 100 - mape*100}
  return res

In [ ]:
def Accuracy(result): #tính toán độ chính xác cho các mô hình học máy với dữ liệu đầu vào nhận từ hàm result bên dưới và tính toán bằng cách sử dụng các loss function ở hàm err bên trên như một công thức
    Accuracy = pd.DataFrame(columns=['Subject', 'MAE', 'MAPE (%)', 'MSE', 'RMSE', 'Accuracy (%)'])
    data = []
    data.append(err('Maths_1_12', result['Maths_1_12'], result['Maths_1_12_pred'])) #gọi hàm err để tính toán độ chính xác với tham số đầu vào là kết quả thực (Math) và kết quả model dự đoán được (Maths_pred)
    data.append(err('Literature_1_12', result['Literature_1_12'], result['Literature_1_12_pred']))
    data.append(err('Physics_1_12', result['Physics_1_12'], result['Physics_1_12_pred']))
    data.append(err('Chemistry_1_12', result['Chemistry_1_12'], result['Chemistry_1_12_pred']))
    data.append(err('Biology_1_12', result['Biology_1_12'], result['Biology_1_12_pred']))
    data.append(err('History_1_12', result['History_1_12'], result['History_1_12_pred']))
    data.append(err('Geography_1_12', result['Geography_1_12'], result['Geography_1_12_pred']))
    data.append(err('English_1_12', result['English_1_12'], result['English_1_12_pred']))
    data.append(err('Civic Education_1_12', result['Civic Education_1_12'], result['Civic Education_1_12_pred']))

    data.append(err('Maths_2_12', result['Maths_2_12'], result['Maths_2_12_pred'])) #gọi hàm err để tính toán độ chính xác với tham số đầu vào là kết quả thực (Math) và kết quả model dự đoán được (Maths_pred)
    data.append(err('Literature_2_12', result['Literature_2_12'], result['Literature_2_12_pred']))
    data.append(err('Physics_2_12', result['Physics_2_12'], result['Physics_2_12_pred']))
    data.append(err('Chemistry_2_12', result['Chemistry_2_12'], result['Chemistry_2_12_pred']))
    data.append(err('Biology_2_12', result['Biology_2_12'], result['Biology_2_12_pred']))
    data.append(err('History_2_12', result['History_2_12'], result['History_2_12_pred']))
    data.append(err('Geography_2_12', result['Geography_2_12'], result['Geography_2_12_pred']))
    data.append(err('English_2_12', result['English_2_12'], result['English_2_12_pred']))
    data.append(err('Civic Education_2_12', result['Civic Education_2_12'], result['Civic Education_2_12_pred']))
    dfs = [pd.DataFrame(d, index=[0]) for d in data]
    Accuracy = pd.concat(dfs, ignore_index=True) #Ghi các giá trị tính toán được vào biến Accuracy

    return Accuracy

In [ ]:
def predModel(ypred): #hiển thị kết quả
  pred = pd.DataFrame({
    'Toán_1': pd.Series(ypred[:,0]), #gán tất cả các hàng của ypred(tất cả học sinh) và cột thứ 1 của ypred(cột điểm môn toán) cho 'Toán' trong dataframe
    'Văn_1': pd.Series(ypred[:,1]),
    'Lý_1': pd.Series(ypred[:,2]),
    'Hoá_1': pd.Series(ypred[:,3]),
    'Sinh_1': pd.Series(ypred[:,4]),
    'Sử_1': pd.Series(ypred[:,5]),
    'Địa_1': pd.Series(ypred[:,6]),
    'Anh_1': pd.Series(ypred[:,7]),
    'GDCD_1': pd.Series(ypred[:,8]),

    'Toán_2': pd.Series(ypred[:,9]),
    'Văn_2': pd.Series(ypred[:,10]),
    'Lý_2': pd.Series(ypred[:,11]),
    'Hoá_2': pd.Series(ypred[:,12]),
    'Sinh_2': pd.Series(ypred[:,13]),
    'Sử_2': pd.Series(ypred[:,14]),
    'Địa_2': pd.Series(ypred[:,15]),
    'Anh_2': pd.Series(ypred[:,16]),
    'GDCD_2': pd.Series(ypred[:,17])

  })
  pred = pred.astype('float64')
  return pred

In [ ]:
def result(ytest, ypred): #sử dụng như một nơi để lưu dữ liệu, hiển thị thông tin giữa điểm thực tế - điểm dự đoán được
  result = pd.DataFrame({
    'Maths_1_12': pd.Series(ytest[:,0]), #lấy tất cả các hàng (tương ứng với tất cả học sinh), và cột thứ 1 (tương ứng với cột điểm môn toán) sau đó gán tất cả cho Maths để tạo dataframe
    'Maths_1_12_pred': pd.Series(ypred[:, 0]),
    'Literature_1_12': pd.Series(ytest[:,1]),
    'Literature_1_12_pred': pd.Series(ypred[:,1 ]),
    'Physics_1_12': pd.Series(ytest[:,2]),
    'Physics_1_12_pred': pd.Series(ypred[:,2 ]),
    'Chemistry_1_12': pd.Series(ytest[:,3]),
    'Chemistry_1_12_pred': pd.Series(ypred[:, 3]),
    'Biology_1_12': pd.Series(ytest[:,4]),
    'Biology_1_12_pred': pd.Series(ypred[:, 4]),
    'History_1_12': pd.Series(ytest[:,5]),
    'History_1_12_pred': pd.Series(ypred[:, 5]),
    'Geography_1_12': pd.Series(ytest[:,6]),
    'Geography_1_12_pred': pd.Series(ypred[:, 6]),
    'English_1_12': pd.Series(ytest[:,7]),
    'English_1_12_pred': pd.Series(ypred[:, 7]),
    'Civic Education_1_12': pd.Series(ytest[:,8]),
    'Civic Education_1_12_pred': pd.Series(ypred[:, 8]),

    'Maths_2_12': pd.Series(ytest[:,9]),
    'Maths_2_12_pred': pd.Series(ypred[:, 9]),
    'Literature_2_12': pd.Series(ytest[:,10]),
    'Literature_2_12_pred': pd.Series(ypred[:,10 ]),
    'Physics_2_12': pd.Series(ytest[:,11]),
    'Physics_2_12_pred': pd.Series(ypred[:,11 ]),
    'Chemistry_2_12': pd.Series(ytest[:,12]),
    'Chemistry_2_12_pred': pd.Series(ypred[:, 12]),
    'Biology_2_12': pd.Series(ytest[:,13]),
    'Biology_2_12_pred': pd.Series(ypred[:, 13]),
    'History_2_12': pd.Series(ytest[:,14]),
    'History_2_12_pred': pd.Series(ypred[:, 14]),
    'Geography_2_12': pd.Series(ytest[:,15]),
    'Geography_2_12_pred': pd.Series(ypred[:, 15]),
    'English_2_12': pd.Series(ytest[:,16]),
    'English_2_12_pred': pd.Series(ypred[:, 16]),
    'Civic Education_2_12': pd.Series(ytest[:,17]),
    'Civic Education_2_12_pred': pd.Series(ypred[:, 17])

  })
  for col in result.columns: result[col] = result[col].apply(lambda x: 10 if x > 10 else x)
  return result

# Split Dataset

In [ ]:
split = 0.8 #80%
X = df.drop(columns=['Maths_1_12', 'Literature_1_12',
       'Physics_1_12', 'Chemistry_1_12', 'Biology_1_12', 'History_1_12',
       'Geography_1_12', 'English_1_12', 'Civic Education_1_12', 'Maths_2_12',
       'Literature_2_12', 'Physics_2_12', 'Chemistry_2_12', 'Biology_2_12',
       'History_2_12', 'Geography_2_12', 'English_2_12',
       'Civic Education_2_12']) #lấy dữ liệu để dùng làm X, chỉ lấy điểm của 4 học kỳ, df.drop nghĩa là sẽ bỏ các cột được liệt kê

#những cột bị drop ở bên trên sẽ được gán riêng như bên dưới và dùng làm Y
Y = df.iloc[:, 36:]

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.2, random_state= 42)

xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((1005, 36), (1005, 18), (252, 36), (252, 18))

In [ ]:
xtrain = xtrain.values
ytrain = ytrain.values
xtest = xtest.values
ytest = ytest.values

In [ ]:
sc = MinMaxScaler(feature_range = (0,1))
sc1 = MinMaxScaler(feature_range = (0,1))
sc2 = MinMaxScaler(feature_range = (0,1))
sc3 = MinMaxScaler(feature_range = (0,1))
xtrain = sc.fit_transform(xtrain)
ytrain = sc1.fit_transform(ytrain)
xtest = sc2.fit_transform(xtest)
temp = sc3.fit_transform(ytest)

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

modelLR = LinearRegression().fit(xtrain, ytrain) #huấn luyện

predLR = modelLR.predict(xtest) #dự đoán trên tập kiểm tra (xtest)

print("predLR size: ", predLR.shape)
print("ytest size: ", ytest.shape)

predLR size:  (252, 18)
ytest size:  (252, 18)


In [ ]:
predLR

array([[0.84044051, 0.77046239, 0.77772257, ..., 0.78698509, 0.68987934,
        0.77946709],
       [0.67010705, 0.61211445, 0.58694008, ..., 0.68771793, 0.49853462,
        0.65272916],
       [0.40714411, 0.47415953, 0.29172528, ..., 0.61639919, 0.31440632,
        0.60064229],
       ...,
       [0.50030045, 0.50322198, 0.42884741, ..., 0.57409045, 0.44291346,
        0.53765593],
       [0.74580892, 0.65757254, 0.67494389, ..., 0.77875342, 0.70941218,
        0.75069355],
       [0.6651955 , 0.57368749, 0.45371555, ..., 0.66015729, 0.4282644 ,
        0.58316591]])

In [ ]:
predLR = sc3.inverse_transform(predLR)

In [ ]:
resultLR = result(ytest, predLR) #lưu kq bằng hàm result và hiển thị
resultLR

,Maths_1_12,Maths_1_12_pred,Literature_1_12,Literature_1_12_pred,Physics_1_12,Physics_1_12_pred,Chemistry_1_12,Chemistry_1_12_pred,Biology_1_12,Biology_1_12_pred,...,Biology_2_12,Biology_2_12_pred,History_2_12,History_2_12_pred,Geography_2_12,Geography_2_12_pred,English_2_12,English_2_12_pred,Civic Education_2_12,Civic Education_2_12_pred
0,8.4,8.658599,8.1,8.167081,6.8,8.621880,9.6,9.240597,7.3,7.973650,...,8.0,8.672201,7.2,8.582245,9.0,8.813624,9.0,8.156360,9.6,9.260455
1,7.3,7.653632,7.7,7.454515,7.4,7.439028,8.5,8.045916,6.3,6.724809,...,8.0,8.406655,7.7,8.430611,8.4,8.307361,7.6,7.142233,9.2,8.892915
2,6.7,6.102150,7.5,6.833718,6.0,5.608697,7.6,6.483432,6.7,5.060998,...,7.5,7.900210,8.2,8.375459,8.5,7.943636,7.3,6.166354,9.1,8.741863
3,5.8,5.425669,6.1,5.977342,3.8,4.944966,5.0,5.859358,5.1,4.805906,...,6.2,7.115036,6.0,7.336263,7.5,7.110252,5.4,5.207769,7.4,8.054798
4,8.4,7.865669,8.0,7.524854,8.8,8.173073,8.2,8.468861,7.5,7.738703,...,6.5,7.067375,7.0,6.927741,7.7,7.398344,6.1,5.268843,8.9,8.256773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,6.0,7.157844,6.9,7.332225,6.6,7.092699,9.3,7.870838,5.7,6.924787,...,7.6,7.860781,7.3,7.886569,7.1,7.743317,7.6,6.778166,8.9,8.475310
248,8.1,6.822519,6.7,7.076902,5.4,6.061030,6.8,6.813170,6.5,6.005650,...,6.6,7.615009,8.5,7.948964,8.4,8.054914,5.5,6.119137,9.1,9.108635
249,5.7,6.651773,7.8,6.964499,5.6,6.458854,8.0,7.709172,6.8,6.555552,...,7.0,7.997409,7.0,7.886946,6.6,7.727861,7.2,6.847441,7.0,8.559202
250,8.3,8.100273,6.8,7.659076,8.1,7.984652,9.0,8.725723,7.6,7.447333,...,8.4,8.803350,9.2,8.906721,9.9,8.771642,7.6,8.259885,8.7,9.177011


In [ ]:
Accuracy(resultLR) #tính toán độ chính xác của model theo dự đoán về từng môn

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths_1_12,0.636986,9.495711,0.657737,0.811010,90.504289
1,Literature_1_12,0.463264,6.667987,0.348468,0.590312,93.332013
2,Physics_1_12,0.725858,11.190926,0.866070,0.930629,88.809074
3,Chemistry_1_12,0.589584,8.155823,0.648421,0.805246,91.844177
4,Biology_1_12,0.674472,10.202934,0.710310,0.842799,89.797066
5,History_1_12,0.601867,8.532843,0.589797,0.767982,91.467157
6,Geography_1_12,0.671027,9.424610,0.776950,0.881448,90.575390
7,English_1_12,0.630128,9.324662,0.708301,0.841606,90.675338
8,Civic Education_1_12,0.480838,5.981699,0.385578,0.620950,94.018301
9,Maths_2_12,0.529461,6.745875,0.470388,0.685848,93.254125


In [ ]:
predModel(predLR) #hiển thị kết quả dự đoán

,Toán_1,Văn_1,Lý_1,Hoá_1,Sinh_1,Sử_1,Địa_1,Anh_1,GDCD_1,Toán_2,Văn_2,Lý_2,Hoá_2,Sinh_2,Sử_2,Địa_2,Anh_2,GDCD_2
0,8.658599,8.167081,8.621880,9.240597,7.973650,8.646052,9.172354,8.246174,9.282214,8.759808,7.881493,7.930254,9.184984,8.672201,8.582245,8.813624,8.156360,9.260455
1,7.653632,7.454515,7.439028,8.045916,6.724809,7.667793,8.271721,6.426916,8.341603,8.415465,8.150362,8.108835,8.745320,8.406655,8.430611,8.307361,7.142233,8.892915
2,6.102150,6.833718,5.608697,6.483432,5.060998,6.809993,7.211547,4.877403,7.512260,7.294221,7.866846,6.963626,7.738424,7.900210,8.375459,7.943636,6.166354,8.741863
3,5.425669,5.977342,4.944966,5.859358,4.805906,6.398837,7.028074,4.429411,6.980465,6.189697,6.386396,5.868667,7.019045,7.115036,7.336263,7.110252,5.207769,8.054798
4,7.865669,7.524854,8.173073,8.468861,7.738703,8.308788,8.746799,7.472785,8.405829,7.540860,7.394934,6.447831,7.389697,7.067375,6.927741,7.398344,5.268843,8.256773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,7.157844,7.332225,7.092699,7.870838,6.924787,7.674460,8.094805,6.841892,8.019677,7.753785,7.882783,7.290873,7.768347,7.860781,7.886569,7.743317,6.778166,8.475310
248,6.822519,7.076902,6.061030,6.813170,6.005650,7.285125,8.192646,5.769371,8.171476,7.282618,6.652551,6.523203,8.019695,7.615009,7.948964,8.054914,6.119137,9.108635
249,6.651773,6.964499,6.458854,7.709172,6.555552,7.422087,7.790654,6.449876,7.720366,7.505508,7.627562,7.420034,8.291540,7.997409,7.886946,7.727861,6.847441,8.559202
250,8.100273,7.659076,7.984652,8.725723,7.447333,8.123241,8.732675,7.385064,8.949796,8.712141,8.063757,8.152018,9.077946,8.803350,8.906721,8.771642,8.259885,9.177011


# MLP

In [ ]:
class MLPModel(Model):
    def __init__(self):
        super().__init__()
        self.flatten = Flatten()
        self.fc1 = Dense(128, activation='relu')
        self.fc2 = Dense(62, activation='relu')
        self.fc3 = Dense(32, activation='relu')
        self.fc4 = Dense(ytrain.shape[1], activation='linear')

    def call(self, inputs):
        features = self.flatten(inputs)
        features = self.fc1(features)
        features = self.fc2(features)
        features = self.fc3(features)
        features = self.fc4(features)
        return features

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)

In [ ]:
MLP = MLPModel()
MLP.compile(optimizer = optimizer, loss = 'mean_squared_error')

In [ ]:
MLP.fit(xtrain, ytrain, epochs = 200, batch_size = 64, callbacks = [history])

Epoch 1/200
16/16 [==============================] - 1s 2ms/step - loss: 0.0766
Epoch 2/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0269
Epoch 3/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0233
Epoch 4/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0221
Epoch 5/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0213
Epoch 6/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0198
Epoch 7/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0189
Epoch 8/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0188
Epoch 9/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 10/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0179
Epoch 11/200
16/16 [==============================] - 0s 3ms/step - loss: 0.0181
Epoch 12/200
16/16 [==============================] - 0s 3ms/step - loss: 0.0175
Epoch 13/200
16/16 [=================

In [ ]:
predMLP = MLP.predict(xtest)
predMLP.shape

8/8 [==============================] - 0s 2ms/step


(252, 18)

In [ ]:
predMLP = sc3.inverse_transform(predMLP)

In [ ]:
resultMLP = result(ytest, predMLP)
resultMLP

,Maths_1_12,Maths_1_12_pred,Literature_1_12,Literature_1_12_pred,Physics_1_12,Physics_1_12_pred,Chemistry_1_12,Chemistry_1_12_pred,Biology_1_12,Biology_1_12_pred,...,Biology_2_12,Biology_2_12_pred,History_2_12,History_2_12_pred,Geography_2_12,Geography_2_12_pred,English_2_12,English_2_12_pred,Civic Education_2_12,Civic Education_2_12_pred
0,8.4,8.364823,8.1,8.161304,6.8,7.928596,9.6,8.685622,7.3,7.600595,...,8.0,8.779241,7.2,8.517536,9.0,8.659046,9.0,8.190628,9.6,9.455049
1,7.3,7.494835,7.7,7.482115,7.4,7.228737,8.5,8.050811,6.3,6.587018,...,8.0,8.424298,7.7,8.459870,8.4,8.411477,7.6,7.353269,9.2,8.905313
2,6.7,5.769645,7.5,6.568079,6.0,5.626725,7.6,6.355287,6.7,5.497965,...,7.5,7.607921,8.2,8.350657,8.5,7.578032,7.3,6.160373,9.1,8.725859
3,5.8,5.178923,6.1,6.523662,3.8,4.800131,5.0,5.471317,5.1,5.017689,...,6.2,7.201646,6.0,7.544899,7.5,6.189588,5.4,5.336718,7.4,8.433323
4,8.4,8.377003,8.0,7.797707,8.8,8.396362,8.2,8.609632,7.5,7.786622,...,6.5,7.106504,7.0,7.186122,7.7,7.793266,6.1,5.991056,8.9,8.617714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,6.0,6.313167,6.9,7.122952,6.6,6.183062,9.3,6.964316,5.7,5.854297,...,7.6,7.894018,7.3,7.931058,7.1,7.602377,7.6,6.560989,8.9,8.679946
248,8.1,7.302454,6.7,7.198870,5.4,6.392075,6.8,7.122145,6.5,6.556138,...,6.6,8.018030,8.5,8.484623,8.4,8.391198,5.5,6.158941,9.1,9.425449
249,5.7,6.953036,7.8,7.173558,5.6,6.512904,8.0,7.436383,6.8,6.000318,...,7.0,8.093893,7.0,8.057334,6.6,8.013473,7.2,7.017326,7.0,8.734118
250,8.3,8.227802,6.8,7.737614,8.1,7.785429,9.0,8.523790,7.6,7.404802,...,8.4,8.629550,9.2,8.877820,9.9,8.750964,7.6,7.884120,8.7,9.391997


In [ ]:
Accuracy(resultMLP)

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths_1_12,0.620254,9.235817,0.646731,0.804196,90.764183
1,Literature_1_12,0.483102,7.046370,0.391043,0.625334,92.953630
2,Physics_1_12,0.690251,10.677831,0.841885,0.917543,89.322169
3,Chemistry_1_12,0.625512,8.571466,0.677620,0.823177,91.428534
4,Biology_1_12,0.630134,9.558918,0.633189,0.795731,90.441082
5,History_1_12,0.555232,7.847022,0.494407,0.703141,92.152978
6,Geography_1_12,0.653439,9.203644,0.754368,0.868543,90.796356
7,English_1_12,0.639039,9.590558,0.719906,0.848473,90.409442
8,Civic Education_1_12,0.476769,6.013775,0.404199,0.635766,93.986225
9,Maths_2_12,0.588064,7.449356,0.574820,0.758169,92.550644


# LSTM

In [ ]:
xtrain_timesteps = xtrain.reshape((xtrain.shape[0],1,xtrain.shape[1]))
xtest_timesteps = xtest.reshape((xtest.shape[0],1,xtest.shape[1]))

In [ ]:
LSTM = tf.keras.layers.LSTM

In [ ]:
class LSTMModel(Model):
  def __init__(self):
    super().__init__()
    self.LSTM1 = LSTM(128, activation = 'relu', return_sequences = True)
    self.LSTM2 = LSTM(64, activation = 'relu', return_sequences= True)
    self.LSTM3 = LSTM(32, activation = 'relu', return_sequences= False)
    self.fc1 = Dense(ytrain.shape[1], activation = 'linear')

  def call(self, inputs):
    features = self.LSTM1(inputs)
    features = self.LSTM2(features)
    features = self.LSTM3(features)
    features = self.fc1(features)
    return features


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)

In [ ]:
LSTMmodel = LSTMModel()
LSTMmodel.compile(optimizer = optimizer, loss = 'mean_squared_error')

In [ ]:
LSTMmodel.fit(xtrain_timesteps, ytrain, epochs = 200, batch_size = 128, callbacks = [history])

Epoch 1/200
8/8 [==============================] - 4s 9ms/step - loss: 0.2783
Epoch 2/200
8/8 [==============================] - 0s 9ms/step - loss: 0.0700
Epoch 3/200
8/8 [==============================] - 0s 9ms/step - loss: 0.0292
Epoch 4/200
8/8 [==============================] - 0s 9ms/step - loss: 0.0261
Epoch 5/200
8/8 [==============================] - 0s 9ms/step - loss: 0.0256
Epoch 6/200
8/8 [==============================] - 0s 14ms/step - loss: 0.0249
Epoch 7/200
8/8 [==============================] - 0s 10ms/step - loss: 0.0238
Epoch 8/200
8/8 [==============================] - 0s 9ms/step - loss: 0.0233
Epoch 9/200
8/8 [==============================] - 0s 10ms/step - loss: 0.0231
Epoch 10/200
8/8 [==============================] - 0s 9ms/step - loss: 0.0229
Epoch 11/200
8/8 [==============================] - 0s 9ms/step - loss: 0.0225
Epoch 12/200
8/8 [==============================] - 0s 10ms/step - loss: 0.0222
Epoch 13/200
8/8 [==============================] - 0s 8m

In [ ]:
predLSTM = LSTMmodel.predict(xtest_timesteps)
predLSTM.shape

8/8 [==============================] - 0s 3ms/step


(252, 18)

In [ ]:
predLSTM = sc3.inverse_transform(predLSTM)

In [ ]:
resultLSTM = result(ytest, predLSTM)
resultLSTM

,Maths_1_12,Maths_1_12_pred,Literature_1_12,Literature_1_12_pred,Physics_1_12,Physics_1_12_pred,Chemistry_1_12,Chemistry_1_12_pred,Biology_1_12,Biology_1_12_pred,...,Biology_2_12,Biology_2_12_pred,History_2_12,History_2_12_pred,Geography_2_12,Geography_2_12_pred,English_2_12,English_2_12_pred,Civic Education_2_12,Civic Education_2_12_pred
0,8.4,8.514320,8.1,8.105084,6.8,8.574900,9.6,9.044996,7.3,8.134930,...,8.0,8.674329,7.2,8.393127,9.0,8.561634,9.0,7.866366,9.6,9.293939
1,7.3,7.529807,7.7,7.256706,7.4,7.429479,8.5,8.249212,6.3,6.664231,...,8.0,8.402396,7.7,8.617192,8.4,8.561820,7.6,7.329224,9.2,8.913572
2,6.7,5.984667,7.5,7.040847,6.0,5.661432,7.6,6.609480,6.7,5.534606,...,7.5,8.049370,8.2,8.166122,8.5,7.719816,7.3,6.318915,9.1,8.821320
3,5.8,5.213627,6.1,5.944347,3.8,4.953293,5.0,5.665798,5.1,5.205080,...,6.2,7.287951,6.0,7.112107,7.5,6.499171,5.4,5.253125,7.4,8.138063
4,8.4,7.811282,8.0,7.437020,8.8,7.952581,8.2,8.371498,7.5,7.555623,...,6.5,7.156218,7.0,7.007298,7.7,7.485278,6.1,5.681742,8.9,8.526482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,6.0,6.627539,6.9,6.943101,6.6,6.319995,9.3,7.346066,5.7,5.886491,...,7.6,8.000302,7.3,8.120440,7.1,8.027226,7.6,7.222696,8.9,8.685400
248,8.1,6.737557,6.7,7.550949,5.4,6.131195,6.8,6.979084,6.5,6.069913,...,6.6,8.038462,8.5,7.893972,8.4,7.736097,5.5,6.194193,9.1,9.148173
249,5.7,6.668205,7.8,6.795445,5.6,6.401435,8.0,7.351676,6.8,5.927996,...,7.0,7.981395,7.0,8.062121,6.6,7.920949,7.2,6.915717,7.0,8.607267
250,8.3,7.986859,6.8,7.608266,8.1,7.989879,9.0,8.724781,7.6,7.233739,...,8.4,8.655008,9.2,8.854364,9.9,8.870431,7.6,8.097878,8.7,9.110509


In [ ]:
Accuracy(resultLSTM)

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths_1_12,0.640140,9.363944,0.656361,0.810161,90.636056
1,Literature_1_12,0.479817,6.966534,0.368077,0.606694,93.033466
2,Physics_1_12,0.719571,11.134180,0.885527,0.941025,88.865820
3,Chemistry_1_12,0.609686,8.466755,0.661536,0.813348,91.533245
4,Biology_1_12,0.637596,9.690755,0.675647,0.821977,90.309245
5,History_1_12,0.604409,8.629920,0.605997,0.778458,91.370080
6,Geography_1_12,0.636670,8.955892,0.736481,0.858185,91.044108
7,English_1_12,0.650945,9.807242,0.750464,0.866293,90.192758
8,Civic Education_1_12,0.473935,5.897583,0.363618,0.603007,94.102417
9,Maths_2_12,0.546197,6.907898,0.503914,0.709869,93.092102


# Save Trained Models

In [ ]:
import pickle
import torch

In [ ]:
with open('LR_10_11_12.pkl', 'wb') as fileLR:
    pickle.dump(modelLR, fileLR)

In [ ]:
with open('MLP_10_11_12.pkl', 'wb') as fileMLP:
    pickle.dump(MLP, fileMLP)

In [ ]:
with open('LSTM_10_11_12.pkl', 'wb') as fileLSTM:
    pickle.dump(LSTMmodel, fileLSTM)